## Data Inspection

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
dt = pd.read_csv('/Users/jingyichen/machinelearning/a-mimic/final_patient_data (4).csv')

In [4]:
dt.head()

,subject_id,gender,insurance,language,religion,marital_status,ethnicity,admission_count,adm_age,los_adm,last_admission_location,last_admission_type,icu_count,icu_age,los_icu,mechanical_ventilation,VASOPRESSOR_USE,death_age,hospital_expired
0,2,M,Private,NaN,NOT SPECIFIED,NaN,ASIAN,1,0.0,3.8639,PHYS REFERRAL/NORMAL DELI,NEWBORN,1,0.0,0.0918,0.0,0.0,NaN,0
1,3,M,Medicare,NaN,CATHOLIC,MARRIED,WHITE,1,76.5,10.7847,EMERGENCY ROOM ADMIT,EMERGENCY,1,76.5,6.0646,1.0,1.0,NaN,0
2,4,F,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,1,47.8,7.7590,EMERGENCY ROOM ADMIT,EMERGENCY,1,47.8,1.6785,1.0,0.0,NaN,0
3,5,M,Private,NaN,BUDDHIST,NaN,ASIAN,1,0.0,2.3222,PHYS REFERRAL/NORMAL DELI,NEWBORN,1,0.0,0.0844,0.0,0.0,NaN,0
4,6,F,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,1,65.9,16.3646,PHYS REFERRAL/NORMAL DELI,ELECTIVE,1,65.9,3.6729,1.0,0.0,NaN,0


In [5]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46520 entries, 0 to 46519
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               46520 non-null  int64  
 1   gender                   46520 non-null  object 
 2   insurance                46520 non-null  object 
 3   language                 25209 non-null  object 
 4   religion                 46086 non-null  object 
 5   marital_status           36757 non-null  object 
 6   ethnicity                46520 non-null  object 
 7   admission_count          46520 non-null  int64  
 8   adm_age                  46520 non-null  float64
 9   los_adm                  46520 non-null  float64
 10  last_admission_location  46520 non-null  object 
 11  last_admission_type      46520 non-null  object 
 12  icu_count                46520 non-null  int64  
 13  icu_age                  46467 non-null  float64
 14  los_icu               

In [7]:
# Check the data types of the columns

# Check the number of values greater than 120 in icu_age
count_above_120_icu_age = (dt['icu_age'] > 120).sum()
print(f"Number of values in icu_age > 120: {count_above_120_icu_age}")

# Check the number of values greater than 120 in death_age
count_above_120_death_age = (dt['death_age'] > 120).sum()
print(f"Number of values in death_age > 120: {count_above_120_death_age}")

# Check for negative and abnormal values in los_adm
count_negative_los_adm = (dt['los_adm'] < 0).sum()
print(f"Number of negative values in los_adm: {count_negative_los_adm}")

# Check the number of missing values
count_null_icu_age = dt['icu_age'].isnull().sum()
count_null_death_age = dt['death_age'].isnull().sum()

print(f"Number of missing values in icu_age: {count_null_icu_age}")
print(f"Number of missing values in death_age: {count_null_death_age}")



Number of values in icu_age > 120: 1990
Number of values in death_age > 120: 549
Number of negative values in los_adm: 92
Number of missing values in icu_age: 53
Number of missing values in death_age: 40707


In [8]:
# Keep rows where death_age <= 150, also keep NaN values
dt = dt[(dt['death_age'] <= 150) | (dt['death_age'].isnull())]

# Check the number of NaN values in the 'death_age' column
print(f"Number of NaN values in the 'death_age' column: {dt['death_age'].isnull().sum()}")



Number of NaN values in the 'death_age' column: 40707


In [9]:
# 1. **Age-related features**
# Remove outliers that exceed a reasonable range
dt = dt[dt['adm_age'] <= 100]
dt = dt[dt['icu_age'] <= 100]

# 2. **Admission count and ICU count**
dt = dt[(dt['admission_count'] <= 10) & (dt['icu_count'] <= 10)]

# 3. **Length of stay: los_adm & los_icu**
# Remove negative values in `los_adm`
dt = dt[dt['los_adm'] >= 0]

# For los_adm, remove only right-side outliers
Q3_los_adm = dt['los_adm'].quantile(0.75)
IQR_los_adm = Q3_los_adm - dt['los_adm'].quantile(0.25)
right_bound_adm = Q3_los_adm + 1.5 * IQR_los_adm
dt = dt[dt['los_adm'] <= right_bound_adm]

# For los_icu, remove only right-side outliers
Q3_los_icu = dt['los_icu'].quantile(0.75)
IQR_los_icu = Q3_los_icu - dt['los_icu'].quantile(0.25)
right_bound_icu = Q3_los_icu + 1.5 * IQR_los_icu
dt = dt[dt['los_icu'] <= right_bound_icu]


In [10]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36637 entries, 0 to 46519
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               36637 non-null  int64  
 1   gender                   36637 non-null  object 
 2   insurance                36637 non-null  object 
 3   language                 20598 non-null  object 
 4   religion                 36306 non-null  object 
 5   marital_status           29597 non-null  object 
 6   ethnicity                36637 non-null  object 
 7   admission_count          36637 non-null  int64  
 8   adm_age                  36637 non-null  float64
 9   los_adm                  36637 non-null  float64
 10  last_admission_location  36637 non-null  object 
 11  last_admission_type      36637 non-null  object 
 12  icu_count                36637 non-null  int64  
 13  icu_age                  36637 non-null  float64
 14  los_icu                  36

### Handle NA in death_age

In [11]:
# Step 1: Create a binary indicator for whether death_age is missing
dt['death_age_missing'] = dt['death_age'].isnull().astype(int)

# Step 2: Use both 'death_age' and 'death_age_missing' as features
# Check the result
print(dt[['death_age', 'death_age_missing']].head())

# Optional: Check the distribution of missing values
print(dt['death_age_missing'].value_counts())


   death_age  death_age_missing
0        NaN                  1
1        NaN                  1
2        NaN                  1
3        NaN                  1
4        NaN                  1
death_age_missing
1    32946
0     3691
Name: count, dtype: int64


In [12]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36637 entries, 0 to 46519
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               36637 non-null  int64  
 1   gender                   36637 non-null  object 
 2   insurance                36637 non-null  object 
 3   language                 20598 non-null  object 
 4   religion                 36306 non-null  object 
 5   marital_status           29597 non-null  object 
 6   ethnicity                36637 non-null  object 
 7   admission_count          36637 non-null  int64  
 8   adm_age                  36637 non-null  float64
 9   los_adm                  36637 non-null  float64
 10  last_admission_location  36637 non-null  object 
 11  last_admission_type      36637 non-null  object 
 12  icu_count                36637 non-null  int64  
 13  icu_age                  36637 non-null  float64
 14  los_icu                  36

## Feature Engineering

In [13]:
# language -- ENGL, OTHER
# Set all values in the 'language' column that are not 'ENGL' to 'OTHER'
dt['language'] = dt['language'].apply(lambda x: 'ENGL' if x == 'ENGL' else 'OTHER')

# Check the result
print(dt['language'].value_counts())


language
OTHER    18616
ENGL     18021
Name: count, dtype: int64


In [14]:
# Find the top 5 most frequent religions
top5_religions = dt['religion'].value_counts().nlargest(5).index

# Set all other values to 'OTHER'
dt['religion'] = dt['religion'].apply(lambda x: x if x in top5_religions else 'OTHER')

# View the result after processing
print(dt['religion'].value_counts())



religion
CATHOLIC             12560
NOT SPECIFIED         7855
UNOBTAINABLE          5772
PROTESTANT QUAKER     4079
OTHER                 3673
JEWISH                2698
Name: count, dtype: int64


In [15]:
def simplify_ethnicity(x):
    x = str(x).upper() 
    if 'WHITE' in x:
        return 'WHITE'
    elif 'BLACK' in x:
        return 'BLACK'
    elif 'ASIAN' in x:
        return 'ASIAN'
    elif 'HISPANIC' in x or 'LATINO' in x:
        return 'HISPANIC'
    elif 'PORTUGUESE' in x or 'EUROPEAN' in x:
        return 'WHITE'
    elif 'UNABLE TO OBTAIN' in x or 'UNKNOWN' in x or 'PATIENT DECLINED' in x:
        return 'UNKNOWN'
    else:
        return 'OTHER'



In [16]:
dt['ethnicity'] = dt['ethnicity'].apply(simplify_ethnicity)

print(dt['ethnicity'].value_counts())


ethnicity
WHITE       25472
UNKNOWN      4187
BLACK        3082
ASIAN        1402
HISPANIC     1330
OTHER        1164
Name: count, dtype: int64


In [17]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36637 entries, 0 to 46519
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               36637 non-null  int64  
 1   gender                   36637 non-null  object 
 2   insurance                36637 non-null  object 
 3   language                 36637 non-null  object 
 4   religion                 36637 non-null  object 
 5   marital_status           29597 non-null  object 
 6   ethnicity                36637 non-null  object 
 7   admission_count          36637 non-null  int64  
 8   adm_age                  36637 non-null  float64
 9   los_adm                  36637 non-null  float64
 10  last_admission_location  36637 non-null  object 
 11  last_admission_type      36637 non-null  object 
 12  icu_count                36637 non-null  int64  
 13  icu_age                  36637 non-null  float64
 14  los_icu                  36

In [18]:
# Replace all null values in 'marital_status' with 'UNKNOWN'
dt['marital_status'] = dt['marital_status'].fillna('UNKNOWN')

# Merge 'UNKNOWN' and 'UNKNOWN (DEFAULT)' into a single 'UNKNOWN' category
dt['marital_status'] = dt['marital_status'].replace({'UNKNOWN': 'UNKNOWN', 'UNKNOWN (DEFAULT)': 'UNKNOWN'})

# Check the result
print(dt['marital_status'].value_counts())


marital_status
MARRIED         15337
SINGLE           7994
UNKNOWN          7272
WIDOWED          3767
DIVORCED         1943
SEPARATED         312
LIFE PARTNER       12
Name: count, dtype: int64


In [19]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36637 entries, 0 to 46519
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               36637 non-null  int64  
 1   gender                   36637 non-null  object 
 2   insurance                36637 non-null  object 
 3   language                 36637 non-null  object 
 4   religion                 36637 non-null  object 
 5   marital_status           36637 non-null  object 
 6   ethnicity                36637 non-null  object 
 7   admission_count          36637 non-null  int64  
 8   adm_age                  36637 non-null  float64
 9   los_adm                  36637 non-null  float64
 10  last_admission_location  36637 non-null  object 
 11  last_admission_type      36637 non-null  object 
 12  icu_count                36637 non-null  int64  
 13  icu_age                  36637 non-null  float64
 14  los_icu                  36

In [20]:
dt.to_csv('/Users/jingyichen/Desktop/datapre.csv', index=False)

## Before & After

In [22]:
import pandas as pd

# Load the data
dt_bf = pd.read_csv("/Users/jingyichen/machinelearning/a-mimic/final_patient_data (4).csv")  # Replace with your file path

# View the dataset size (number of rows × number of columns)
print(f"Dataset shape: {dt_bf.shape}")  # Output format: (rows, columns)

# View dataset structure (column names, data types, non-null counts)
print("\nDataset structure:")
print(dt_bf.info())


Dataset shape: (46520, 19)

Dataset structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46520 entries, 0 to 46519
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subject_id               46520 non-null  int64  
 1   gender                   46520 non-null  object 
 2   insurance                46520 non-null  object 
 3   language                 25209 non-null  object 
 4   religion                 46086 non-null  object 
 5   marital_status           36757 non-null  object 
 6   ethnicity                46520 non-null  object 
 7   admission_count          46520 non-null  int64  
 8   adm_age                  46520 non-null  float64
 9   los_adm                  46520 non-null  float64
 10  last_admission_location  46520 non-null  object 
 11  last_admission_type      46520 non-null  object 
 12  icu_count                46520 non-null  int64  
 13  icu_age                  4646

In [23]:
import pandas as pd

def compare_datasets(original_path, processed_path):
    df_orig = pd.read_csv(original_path)
    df_proc = pd.read_csv(processed_path)

    print("📐 Dataset Shape Comparison:")
    print(f"Original shape : {df_orig.shape}")
    print(f"Processed shape: {df_proc.shape}\n")

    print("📉 Missing Value Comparison (Top 10 columns with missing values):")
    orig_missing = df_orig.isnull().sum()
    proc_missing = df_proc.isnull().sum()
    missing_df = pd.DataFrame({
        "Original Missing": orig_missing,
        "Processed Missing": proc_missing
    }).query("`Original Missing` > 0 or `Processed Missing` > 0")
    print(missing_df.sort_values("Original Missing", ascending=False).head(10), "\n")

    print("🧬 Data Type Changes:")
    dtype_diff = pd.concat([df_orig.dtypes, df_proc.dtypes], axis=1, keys=['Original', 'Processed'])
    dtype_diff = dtype_diff[dtype_diff['Original'] != dtype_diff['Processed']]
    print(dtype_diff if not dtype_diff.empty else "No data type changes.\n")

    print("\n📌 Column Comparison:")
    orig_cols = set(df_orig.columns)
    proc_cols = set(df_proc.columns)
    print("Dropped columns :", orig_cols - proc_cols)
    print("Added columns   :", proc_cols - orig_cols)


compare_datasets("/Users/jingyichen/machinelearning/a-mimic/final_patient_data (4).csv", "/Users/jingyichen/Desktop/datapre.csv")

📐 Dataset Shape Comparison:
Original shape : (46520, 19)
Processed shape: (36637, 20)

📉 Missing Value Comparison (Top 10 columns with missing values):
                        Original Missing  Processed Missing
death_age                        40707.0              32946
language                         21311.0                  0
marital_status                    9763.0                  0
religion                           434.0                  0
VASOPRESSOR_USE                     53.0                  0
icu_age                             53.0                  0
los_icu                             53.0                  0
mechanical_ventilation              53.0                  0 

🧬 Data Type Changes:
                  Original Processed
death_age_missing      NaN     int64

📌 Column Comparison:
Dropped columns : set()
Added columns   : {'death_age_missing'}
